In [1]:
from deepmd import DeepMD
import numpy as np

data = DeepMD("IVDW_11", "0")

In [2]:
print(type(data.model))
for name, datas in data.model.__dict__.items():
    print(f"{name}: {datas.shape}")

<class 'deepmd.DeepMDModel'>
box: (2000, 9)
coord: (2000, 54, 3)
energy: (2000, 1)
force: (2000, 54, 3)
type: (54,)
virial: (2000, 9)


In [3]:
print(data.model)

DeepMDModel(box=array([[ 1.05600000e+01,  1.01924569e-15,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  3.00000000e+01],
       [ 1.05600000e+01,  1.01924569e-15,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  3.00000000e+01],
       [ 1.05600000e+01,  1.01924569e-15,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  3.00000000e+01],
       ...,
       [ 1.04559642e+01,  0.00000000e+00,  0.00000000e+00, ...,
        -1.98974839e-01,  4.28366867e-01,  3.00611558e+01],
       [ 1.04559642e+01,  0.00000000e+00,  0.00000000e+00, ...,
        -1.98974839e-01,  4.28366867e-01,  3.00611558e+01],
       [ 1.04559642e+01,  0.00000000e+00,  0.00000000e+00, ...,
        -1.98974839e-01,  4.28366867e-01,  3.00611558e+01]]), coord=array([[[ 1.76000305e+00,  1.01613471e+00,  2.07348400e+01],
        [ 2.55216979e-06,  4.06454442e+00,  2.07348400e+01],
        [-1.75999795e+00,  7.11295413e+00,  2.07348400e+01],
        ...,
        [ 7.04000100e+00